# 0. Import các thư viện

In [1]:
import requests
import pandas as pd
from collections import defaultdict
from bs4 import BeautifulSoup

# 1. Phân tích cách cào trên 1 trang và cào từng game

In [2]:
# Hàm này nhận vào index và trả về địa chỉ của trang

# def game_page(page):
#     return f'https://vi.y8.com/?page={page}'

game_page = lambda page: f'https://vi.y8.com/?page={page}'

In [3]:
game_page(1)

'https://vi.y8.com/?page=1'

In [4]:
# Gửi request lấy thông tin trang 1 và parse html
res = requests.get(game_page(1))
soup = BeautifulSoup(res.content, 'html.parser')

In [5]:
# Lấy all link game trong 1 trang

# games_url = []
# for x in soup.select('div.thumbarea > a'):
#     games_url.append(x['href'])
# games_url[0]

games_url = list(map(lambda x: x['href'], soup.select('div.thumbarea > a')))
games_url[0]

'https://vi.y8.com/games/8ball_online'

In [6]:
# Gửi request lấy thông tin 1 game cụ thể
res = requests.get(games_url[0])
soup = BeautifulSoup(res.content, 'html.parser')

In [7]:
# Phần thông tin chứa tên và sub
left_part = soup.select('div.left-part')[-1]

In [8]:
# Lấy tên game
name = left_part.select_one('h1').text.strip()
name

'8Ball Online'

In [9]:
# Lấy sub
sub = left_part.select_one('span.sub-infos-line > span').text.split(' ')[0]
sub

'2,077,678'

In [10]:
rating = soup.select_one('div.rating').text.strip()
rating

'66.6%'

In [11]:
desc = soup.select_one('div.ltr.description').text.strip()
desc

'Challenge a real opponent from all around the world in this multiplayer version of the popular sports game! 8 Ball Pool is played with 15 numbered balls from 1 to 15 and one white cue ball. To win a match, you have to be the first to pocket the black 8 ball legally. Simply hit play to start the matching process and find an opponent. After the initial break, one of the players has to pocket the solid-colored balls from 1 to 7 while the other tries to remove all striped balls from 9 to 15 from the table. Players are not allowed to pocket the 8 ball until they have completely pocketed their group of striped or solid balls. Try to sink as many of your balls in a row as you can - as soon as you scratch or miss a pocket, it is the other player’s turn!'

In [12]:
date = soup.select('div.item > span.data')[-1].text.strip()
date

'16 Jul 2019'

In [13]:
# Hàm này lấy toàn bộ tag của 1 trò chơi, dòng 2 dùng để nối các chuỗi tag thành 1 chuỗi
# sử dụng dấu phẩy (,)

tmp = list(map(lambda x: x.text, soup.select('div.col-md-12.tags-list > a')))
tags = ', '.join(tmp)
print('list tags string:', tmp)
print('a tags string:', tags)

list tags string: ['1 Người chơi', 'Thể thao', 'Nhiều người chơi', 'Kỹ năng Dùng chuột', 'Bi-a lỗ', 'Timing', 'Android', 'HTML5', 'Miễn phí', 'Di động', 'iPhone', 'iPad', 'Cảm ứng']
a tags string: 1 Người chơi, Thể thao, Nhiều người chơi, Kỹ năng Dùng chuột, Bi-a lỗ, Timing, Android, HTML5, Miễn phí, Di động, iPhone, iPad, Cảm ứng


# 2. Toàn bộ code cào

In [14]:
# Sử dụng defaultdict với từng phần tử là một list
# hiểu đơn giản là dict với key là một string và value là một list 
games = defaultdict(list)

# 17 - 1 = 16 trang * 64 = 1024 trò (hoặc có thể đếm số lượng riêng vì có thể 
# có một số game không cào được, nên sử dụng try - except (khuyến khích tìm hiểu))
for page in range(1, 17):
    all_games = requests.get(game_page(page))
    soup = BeautifulSoup(all_games.content, 'html.parser')

    games_url = list(map(lambda x: x['href'], soup.select('div.thumbarea > a')))

    for game in games_url:
        res = requests.get(game)
        soup = BeautifulSoup(res.content, 'html.parser')

        left_part = soup.select('div.left-part')[-1]
        name = left_part.select_one('h1').text.strip()
        subs = left_part.select_one('span.sub-infos-line > span').text.split(' ')[0]
        rate = soup.select_one('div.rating').text.strip()
        desc = soup.select_one('div.ltr.description').text.strip()
        date = soup.select('div.item > span.data')[-1].text.strip()
        temp = list(map(lambda x: x.text, soup.select('div.col-md-12.tags-list > a')))
        tags = ', '.join(temp)

        games['name'].append(name)
        games['subs'].append(subs)
        games['rate'].append(rate)
        games['desc'].append(desc)
        games['date'].append(date)
        games['tags'].append(tags)
        games['link'].append(game)

    print(f'Page success: {page}, number of games: {len(games["name"])}')

# Khúc này để xuất data ra file csv, dữ liệu mỗi lần cào có thể khác nhau
# do bên Y8 người ta sắp xếp lại xu hướng game
pd.DataFrame(games).to_csv('assets/games_data.csv', index=False, header=True)

Page success: 1, number of games: 64
